In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import pandas_ta as ta
import pickle
from sklearn.metrics import mean_squared_error

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
import os
from datetime import datetime, timedelta
sys.path.insert(1, '/home/krystian/Repos/XTB-Trader/utility')
sys.path.insert(1, '/home/krystian/Repos/XTB-Trader')
from XTBClient.api import XTBClient, PERIOD
from utility import utility 

def get_data_for_symbol(symbol):
    c = XTBClient()
    c.login(os.environ.get("XTB_user_num"), os.environ.get("XTB_pass"))
    df = c.get_candles_in_range(symbol, PERIOD.FIFTEEN_MINUTES.value, start=datetime.now() - timedelta(days=365))
    c.logout()
    
    df = df.reset_index()
    df.rename(columns={"timestamp": "date"}, inplace=True)
    
    df['date'] = df['date'].apply(pd.to_datetime)
    df.set_index('date',inplace=True)
    # df.drop([col for col in df.columns if col != "close"], axis=1, inplace=True)
    return df

def import_csv(name):
    df = pd.read_csv(name, sep=';')
    df.columns = ["date","open","high","low","close", "volume"]
    df.date = pd.to_datetime(df.date)
    df.drop(columns="volume", inplace=True)
    df.set_index('date', inplace=True, drop=True)
    return df

In [4]:
def trend_accuracy(real, preds):
    df = pd.DataFrame({"real": real, "preds": preds})
    return len(df[df.real * df.preds > 0]) / len(df)

# Data import

In [5]:
# df = yf.download(tickers = 'EURUSD=x', period='1y', interval='15m')
df = get_data_for_symbol("EURUSD")
df.index.rename("Date", inplace=True)
df.rename(columns={col: col.lower() for col in df.columns}, inplace=True)
df.index.rename(df.index.name.lower(), inplace=True)
df

,open,close,high,low,volume
date,,,,,
2023-01-01 22:00:00,1.07058,1.06932,1.07086,1.06932,187.0
2023-01-01 22:15:00,1.06932,1.06933,1.06935,1.06906,102.0
2023-01-01 22:30:00,1.06931,1.06900,1.06936,1.06857,798.0
2023-01-01 22:45:00,1.06905,1.06944,1.06981,1.06895,1980.0
2023-01-01 23:00:00,1.06922,1.06777,1.06933,1.06773,285.0
...,...,...,...,...,...
2023-07-03 17:45:00,1.09117,1.09079,1.09120,1.09079,1933.0
2023-07-03 18:00:00,1.09080,1.09116,1.09121,1.09080,2238.0
2023-07-03 18:15:00,1.09116,1.09118,1.09120,1.09105,1769.0


In [6]:
df.ta.indicators(as_list=True);

# Additional Indicators

In [7]:
df['RSI']=ta.rsi(df.close, length=15)
df['EMAF']=ta.ema(df.close, length=20)
df['EMAM']=ta.ema(df.close, length=100)
df['EMAS']=ta.ema(df.close, length=150)
df['EMAS2']=ta.ema(df.close, length=120)

df['gain'] = df['close'].diff()
df['future_gain'] = df['gain'].shift(1)
# df['future'] = df['close'].shift(1)
df = pd.concat([df, ta.stoch(df.high, df.low, df.close)], axis=1)

df.dropna(inplace=True)
df

,open,close,high,low,volume,RSI,EMAF,EMAM,EMAS,EMAS2,gain,future_gain,STOCHk_14_3_3,STOCHd_14_3_3
date,,,,,,,,,,,,,,
2023-01-03 11:15:00,1.05270,1.05261,1.05291,1.05191,12215.0,15.644104,1.057125,1.064082,1.066306,1.064679,-0.00010,-0.00035,4.157572,3.550690
2023-01-03 11:30:00,1.05260,1.05314,1.05357,1.05251,10121.0,20.275578,1.056745,1.063865,1.066132,1.064489,0.00053,-0.00010,8.574807,5.260125
2023-01-03 11:45:00,1.05314,1.05279,1.05349,1.05263,11677.0,19.517384,1.056368,1.063646,1.065955,1.064295,-0.00035,0.00053,12.849630,8.527336
2023-01-03 12:00:00,1.05280,1.05326,1.05330,1.05237,12848.0,23.626454,1.056072,1.063440,1.065787,1.064113,0.00047,-0.00035,19.409084,13.611174
2023-01-03 12:15:00,1.05324,1.05395,1.05397,1.05304,11486.0,29.303880,1.055870,1.063252,1.065630,1.063945,0.00069,0.00047,29.993749,20.750821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-03 17:45:00,1.09117,1.09079,1.09120,1.09079,1933.0,48.987178,1.091074,1.090158,1.089952,1.090049,-0.00039,-0.00023,40.555295,40.813118
2023-07-03 18:00:00,1.09080,1.09116,1.09121,1.09080,2238.0,53.284124,1.091082,1.090177,1.089968,1.090067,0.00037,-0.00039,43.553459,42.080420
2023-07-03 18:15:00,1.09116,1.09118,1.09120,1.09105,1769.0,53.510914,1.091091,1.090197,1.089984,1.090085,0.00002,0.00037,43.553459,42.554071


# Train Test split

In [8]:
# train, test = df[:int(len(df)*0.8)], df[int(len(df)*0.8):]
train = df[df.index < df.index.min() + (df.index.max() - df.index.min()) * 0.8]
test = df[df.index >= df.index.min() + (df.index.max() - df.index.min()) * 0.8]

In [9]:
len(train), len(test)

(9847, 2468)

In [10]:
target = 'future_gain'
Y_train = train[target]
X_train = train.drop(target, axis=1)

Y_test = test[target]
X_test = test.drop(target, axis=1)

# Model

In [11]:
class MAModel:
    def __init__(self, means: list):
         self.means = means

    def predict(self, df):
        target = df["close"]

        ma_cols = []
        for i, mean in enumerate(self.means):
            name = "MA_" + str(mean)
            df[name] = target.rolling(window=mean).mean()
            df[name] = df[name].shift(periods=i)
            ma_cols.append(name)

        df["diff"] = df[ma_cols[0]] - df[ma_cols[1]]
        df["diff"] = df["diff"].astype(float)
        df["diff_prev"] = df["diff"].shift(1)

        df.dropna(how="any", inplace=True)
        df.reset_index(drop=True, inplace=True)

        def is_trade(row):
            if row["diff"] >= 0 and row["diff_prev"] < 0:
                return 1
            if row["diff"] <= 0 and row["diff_prev"] > 0:
                return -1

            return 0

        df["is_trade"] = df.apply(is_trade, axis=1)

        # return df.iloc[0]["is_trade"]
        df_trades = df[df.is_trade != 0].copy().reset_index()

        df_trades["delta"] = (df_trades.close.diff() / 0.0001).shift(-1)
        df_trades["gain"] = (
            df_trades["delta"] * df_trades["is_trade"] + 10 * df_trades["is_trade"]
        )    
        
        return df

In [12]:
model = MAModel([10,20])

In [13]:
model.predict(df)

,open,close,high,low,volume,RSI,EMAF,EMAM,EMAS,EMAS2,gain,future_gain,STOCHk_14_3_3,STOCHd_14_3_3,MA_10,MA_20,diff,diff_prev,is_trade
0,1.05546,1.05443,1.05549,1.05395,19476.0,41.249947,1.055924,1.061159,1.063675,1.061982,-0.00102,0.00104,23.201931,30.403960,1.056358,1.055519,0.000838,0.001275,0
1,1.05442,1.05575,1.05583,1.05397,16039.0,47.258884,1.055907,1.061052,1.063570,1.061879,0.00132,-0.00102,27.643179,27.445882,1.056222,1.055584,0.000638,0.000838,0
2,1.05577,1.05557,1.05587,1.05536,12355.0,46.563071,1.055875,1.060944,1.063464,1.061775,-0.00018,0.00132,25.913621,25.586244,1.055874,1.055732,0.000142,0.000638,0
3,1.05560,1.05513,1.05581,1.05497,11585.0,44.834201,1.055804,1.060829,1.063353,1.061665,-0.00044,-0.00018,29.789590,27.782130,1.055649,1.055848,-0.000198,0.000142,-1
4,1.05517,1.05477,1.05534,1.05449,10220.0,43.420907,1.055706,1.060709,1.063240,1.061551,-0.00036,-0.00044,24.363234,26.688815,1.055429,1.055907,-0.000478,-0.000198,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,1.09117,1.09079,1.09120,1.09079,1933.0,48.987178,1.091074,1.090158,1.089952,1.090049,-0.00039,-0.00023,40.555295,40.813118,1.091472,1.091275,0.000197,0.000289,0
12290,1.09080,1.09116,1.09121,1.09080,2238.0,53.284124,1.091082,1.090177,1.089968,1.090067,0.00037,-0.00039,43.553459,42.080420,1.091408,1.091274,0.000134,0.000197,0
12291,1.09116,1.09118,1.09120,1.09105,1769.0,53.510914,1.091091,1.090197,1.089984,1.090085,0.00002,0.00037,43.553459,42.554071,1.091351,1.091295,0.000055,0.000134,0
12292,1.09117,1.09098,1.09117,1.09093,1883.0,50.865202,1.091081,1.090213,1.089997,1.090100,-0.00020,0.00002,43.892308,43.666409,1.091264,1.091290,-0.000026,0.000055,-1


# Saving the model

In [17]:
filename = os.path.join("../", "../", "models", "main_model.pkl")

In [18]:
pickle.dump(model, open(filename, 'wb'))

In [19]:
loaded_model = pickle.load(open(filename, 'rb'))